# Pandas数据预处理与PyTorch的数据加载

## 一、Pandas简单数据预处理
Pandas是python的一个数据分析包，提供了大量能使我们快速便捷地处理数据的函数和方法。
本文以一个简单的样例介绍如何使用pandas处理原始数据并将它们转化为tensor。

首先创建一个csv文件作为原始数据，将数据按行写入：

In [1]:
import os

data_file = 'house_tiny.csv'
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # Column names
    f.write('NA,Pave,127500\n')  # Each row represents a data example
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

接下来使用pandas包中的``read_csv()``方法读取csv文件，读入的数据以DataFrame的形式呈现：

In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       NaN  Pave  127500
2       2.0   NaN  106000
3       4.0   NaN  178100
4       NaN   NaN  140000


查看数据相关属性:

In [3]:
data.shape # 查看行数与列数
data.head() # 查看数据前5行
data.info() # 查看每一列的计数及数据类型等信息
data.describe() # 查看统计信息

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NumRooms  2 non-null      float64
 1   Alley     2 non-null      object 
 2   Price     5 non-null      int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 248.0+ bytes


,NumRooms,Price
count,2.000000,5.000000
mean,3.000000,135820.000000
std,1.414214,26611.783104
min,2.000000,106000.000000
25%,2.500000,127500.000000
50%,3.000000,127500.000000
75%,3.500000,140000.000000
max,4.000000,178100.000000


使用``duplicated()``和``drop_duplicates()``方法可以实现数据去重：

In [4]:
# 查看是否有重复项
'''
参数keep影响返回的Boolean值。
"first": 重复项中第一项设置为False，其他为True
"last": 最后一项为False，其他为True
False: 所有重复项都为True
'''
data.duplicated(keep="first")

0    False
1     True
2    False
3    False
4    False
dtype: bool

In [5]:
# 去除重复项
'''
参数keep的效果与duplicated()相对应，将会删除被设置为True的重复项
'''
data = data.drop_duplicates(keep="first")
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
2       2.0   NaN  106000
3       4.0   NaN  178100
4       NaN   NaN  140000


使用iloc可以根据索引获得数据切片，这里将data的前两列划为inputs，最后一列化为outputs：

In [6]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
print(inputs)

   NumRooms Alley
0       NaN  Pave
2       2.0   NaN
3       4.0   NaN
4       NaN   NaN


数据中存在多个NaN，常用处理缺失数据的方法可以分为两种：
- imputation：用其他值替换缺失值
- deletion：忽略缺失值

对于NumRooms这一列，调用``fillna()``方法，用该列其他数据的平均值补充：

In [7]:
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
2       2.0   NaN
3       4.0   NaN
4       3.0   NaN


对于Alley这一列，使用``get_dummies()``方法，它将``Pave``与``NaN``看作是不同的类别而拆分成两列，结果可以看成是一种one-hot编码。

In [8]:
# Convert categorical variable into dummy/indicator variables
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0           1          0
2       2.0           0          1
3       4.0           0          1
4       3.0           0          1


inputs与outputs中的每一项都是数字，借助numpy可以转化成tensor格式，之后就能方便地进行各种张量运算。

In [9]:
import numpy as np

X, y = np.array(inputs.values), np.array(outputs.values)
X, y

(array([[3., 1., 0.],
        [2., 0., 1.],
        [4., 0., 1.],
        [3., 0., 1.]]),
 array([127500, 106000, 178100, 140000], dtype=int64))

以上只涵盖了pandas使用方法的极小部分内容，关于pandas更详细的使用说明可以阅读官方文档：https://pandas.pydata.org/docs/user_guide/index.html




---

## 二、PyTorch数据加载
PyTorch中关于数据集的相关处理主要涉及两个类，``torch.utils.data.Dataset``和``torch.utils.data.DataLoader``。本文对此进行简单的介绍，更多详细内容可以阅读``torch.utils.data``的官方文档：
- 英文：https://pytorch.org/docs/stable/data.html
- 中文：https://pytorch-cn.readthedocs.io/zh/latest/package_references/data/

### Dataset类

Dataset是一个抽象类，在使用PyTorch时，为了方便读取，我们需要将使用的数据包装为Dataset类。自定义的数据集应该作为它的子类，并且重写``Dataset``的两个成员函数：``__getitem__``与``__len__``。
- ``__getitem__``：必须实现，返回指定key所对应的数据样本。
- ``__len__``：可选，返回数据集的大小。

### Dataloader类

Dataloader类将数据集与采样器结合，返回给定数据集上的一个迭代器。
相比起用简单的for循环来迭代数据，dataloader保留了更多的信息，它使得我们可以按批读取数据、打乱数据顺序、使用多进程并行加载数据等等。

Dataloder完整的参数如下：
```python
class torch.utils.data.DataLoader(
    dataset: torch.utils.data.dataset.Dataset[T_co],  # 传入的数据集
    batch_size: Optional[int] = 1,              # 每个batch的样本数
    shuffle: bool = False,                   # 在每个epoch开始前是否对数据进行重新排序
    sampler: Optional[torch.utils.data.sampler.Sampler[int]] = None,  # 定义从数据集中取样的策略
    batch_sampler: Optional[torch.utils.data.sampler.Sampler[Sequence[int]]] = None,  # 与sampler类似，每次返回一个batch的索引。与batch_size，shuffle，sampler和drop_last互斥
    num_workers: int = 0,                # 加载数据时使用的子进程数，0为不使用多进程
    collate_fn: Callable[List[T], Any] = None,  # 将一个list的样本组合成一个mini-batch
    pin_memory: bool = False,  # 是否在返回前将tensors拷贝到CUDA的固定内存中
    drop_last: bool = False,   # 数据集大小不是batch_size的整数倍时，是否丢弃最后一个不完整的batch
    
    # 以下是使用多进程加载数据时，关于各个worker的参数设置
    timeout: float = 0, 
    worker_init_fn: Callable[int, None] = None, 
    multiprocessing_context=None, generator=None,
    prefetch_factor: int = 2, 
    persistent_workers: bool = False
)
```





PyTorch数据加载可以分为两种情况：
- 加载PyTorch自带数据集
- 加载自己的数据集

### 加载PyTorch自带数据集
在``torchvision.datasets``中，包括了MNIST，Imagenet-12，CIFAR等常用数据集，且所有数据集都是``torch.utils.data.Dataset``的子类。

参考文档：https://pytorch.org/docs/stable/torchvision/datasets.html

以手写数字MNIST数据集为例，运行以下代码，将会完成数据集的导入。

In [12]:
import torch
import torchvision
from PIL import Image

MNIST_set = torchvision.datasets.MNIST(
    root = 'MNIST/MNIST/', # 数据集的路径（MNIST所在的文件夹）
    train = True,   # 从训练集training.pt或测试集test.pt创建数据集
    download = False # True表示从网上下载并放入指定目录，如果已经自己下载了，设置为False
)

试着查看数据，``MNIST_set[0]``包含了一张图片——手写数字5，以及它的标签“5”。

In [13]:
print(MNIST_set[0])
img, label = MNIST_set[0]
print(img)
print(label)
img.show()

(<PIL.Image.Image image mode=L size=28x28 at 0x19459B2B970>, 5)
<PIL.Image.Image image mode=L size=28x28 at 0x19459B2B400>
5


接下来进行Dataloader的实例化。
由于数据中包含了图像，首先需要将他们转换为tensor，否则在之后会报错：``batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>``

可以使用torchvision.transform模块进行转换，该模块提供了一般的图形转换操作类，用于数据处理，通常的预处理操作有：改变图像尺寸，数据增强（随即切片，镜像），数据维度变换等等。
感兴趣的同学可以查阅相关文档：https://pytorch.org/docs/stable/torchvision/transforms.html

In [ ]:
from torchvision import transforms

mytransform = transforms.Compose([
    transforms.ToTensor() # 将PIL Image或numpy.ndarray转换为tensor
    # 还可以增加别的变换
])
MNIST_set = torchvision.datasets.MNIST(
    root = './MNIST',
    train = True,
    download = False,
    transform = mytransform
)

In [ ]:
img, label = MNIST_set[0]
print(img)
print(label)

接下来就可以进行dataloader的构造：

In [ ]:
MNIST_loader = torch.utils.data.DataLoader(
    MNIST_set,
    batch_size = 10,
    shuffle = False,
    num_workers = 2
)

显示部分数据，简单测试是否读取成功：

In [ ]:
for i, (data, label) in enumerate(MNIST_loader):
    img = transforms.ToPILImage()(data[0][0]) # 将tensor转换为图片
    print(label[0]) # 图片对应的数字标签
    img.show()
    break

构造完的dataloader会包含两部分数据：数据集中的图像构成的tensor，图像对应的标签构成的tensor。
dataloader是一个可迭代对象，除了通过for循环访问数据，也可以使用迭代器：

In [ ]:
data_iterator = iter(MNIST_loader)
batch_data = next(data_iterator)
print(batch_data[0]) # image tensor
print(batch_data[1]) # label tensor

### 加载自己的数据集

加载自己的数据集方式也由这两个步骤构成：先创建一个自定义的数据集类继承Dataset，再使用Dataloader读取数据。
```python
from torch.utils.data import Dataset
import pandas as pd

# 定义自己的数据集，继承Dataset类
class MyDataset(Dataset):
    def __init__(self):
        # 做一些初始化处理，例如数据集路径，文件名列表等
        pass
    
    def __getitem__(self, index):
        # 从文件中读取一条数据，做一些处理并返回
        pass
    
    def __len__(self):
        # 返回数据集的大小
        pass
    
mydataset = Mydataset(xxx)
mydataloader = torch.utils.data.DataLoader(mydataset, batch_size = 10, shuffle = True, num_workers = 0)
```

让我们来看看PyTorch教程中的一个样例，需要预先安装scikit-image与pandas。

原文地址：https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

首先从这里下载所需的数据集：https://download.pytorch.org/tutorial/faces.zip
该数据集的内容是关于面部姿势，对每张人脸图像都用了68个标记点进行标注。

In [ ]:
# 导入接下来需要用到的库
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# 忽略warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # 开启交互模式

在下载好的faces文件夹中，有许多人脸的jpg文件，以及含有标记数据的csv文件，它包含了每张图片的文件名以及它对应的各个标记点的x轴y轴坐标，内容格式像这样：
```
image_name,part_0_x,part_0_y,part_1_x,part_1_y,part_2_x, ... ,part_67_x,part_67_y
0805personali01.jpg,27,83,27,98, ... 84,134
1084239450_e76e00b7e7.jpg,70,236,71,257, ... ,128,312
```
使用pandas读取csv文件，获取标记点的数据：

In [ ]:
landmarks_frame = pd.read_csv('faces/face_landmarks.csv')

n = 65 # 先随意取出一个样本看看
img_name = landmarks_frame.iloc[n, 0]
landmarks = landmarks_frame.iloc[n, 1:]
landmarks = np.asarray(landmarks)
# 以(N, 2)的数组形式获得标记点，其中N表示标记点的个数。
landmarks = landmarks.astype('float').reshape(-1, 2) # 使用-1表示该维度的长度会被自动计算

print('Image name: {}'.format(img_name))
print('Landmarks shape: {}'.format(landmarks.shape))
print('First 4 Landmarks: {}'.format(landmarks[:4]))

写一个简单的帮助函数来显示图像和标记点，并用它来显示上面获取的那个样本。

In [ ]:
def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=10, marker='.', c='r') # 绘制标记点
    plt.pause(0.001)  # pause a bit so that plots are updated

plt.figure()
show_landmarks(io.imread(os.path.join('faces/', img_name)), landmarks)
plt.show()

接下来构造我们自定义的数据集类。

在``__init__``中读取csv文件，在``__getitem__``中读取图片。在``__getitem__``中读取图片的好处是这样图片不需要一下子全部读进内存，而是每次读取一批，可以充分利用内存空间。

我们的数据集以字典的形式存储：``{'image': image, 'landmarks': landmarks}``。

In [ ]:
class FaceLandmarksDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): 标记点csv文件的路径
            root_dir (string): 存放所有图片的路径
            transform (callable, optional): 可选的transform操作
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        # 根据索引从csv文件获取图片文件名
        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        # 构造字典
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

接下来进行该数据集类的实例化，并查看前四个样本：

In [ ]:
face_dataset = FaceLandmarksDataset(csv_file='faces/face_landmarks.csv',
                                    root_dir='faces/')
fig = plt.figure()

for i in range(len(face_dataset)):
    sample = face_dataset[i]

    print(i, sample['image'].shape, sample['landmarks'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()
    ax.set_title('Sample #{}'.format(i))
    ax.axis('off')
    show_landmarks(**sample)

    if i == 3:
        plt.show()
        break

从上面展示的四个样本可以看出，这些样本的大小各不相同，大部分的神经网络都要求图片以固定的尺寸输入网络，我们最好先进行一些预处理。创建三种变换：
- ``Rescale``：修改图片尺寸
- ``RandomCrop``：随机裁剪图片，是一种数据增强方法
- ``ToTensor``：从numpy images转换为torch images

这里将它们写成可调用的类：

In [ ]:
class Rescale(object):
    """将样本中图片修改为规定的尺寸.

    参数:
        output_size (int 或者 tuple): 要求的输出尺寸. 如果是tuple, 输出和output_size匹配。
如果是int, 图片的短边和output_size匹配，长边按比例缩放。
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        h, w = image.shape[:2]
        # 缩放处理
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))

        # 对landmarks来说h和w需要交换位置，因为对图片来说，x和y分别是第1维和第0维
        landmarks = landmarks * [new_w / w, new_h / h]

        return {'image': img, 'landmarks': landmarks}


class RandomCrop(object):
    """随机裁剪图片.

    参数:
        output_size (tuple or int): 期望的输出尺寸. 如果是int, 做正方形裁剪.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        h, w = image.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        image = image[top: top + new_h,
                      left: left + new_w]

        landmarks = landmarks - [left, top]

        return {'image': image, 'landmarks': landmarks}


class ToTensor(object):
    """将ndarrays转化为Tensors."""

    def __call__(self, sample):
        image, landmarks = sample['image'], sample['landmarks']

        # 交换颜色通道，因为
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image),
                'landmarks': torch.from_numpy(landmarks)}

接下来，我们就可以实例化一个经过预处理的数据集，所有样本的大小都得到统一：

In [ ]:
transformed_dataset = FaceLandmarksDataset(csv_file='faces/face_landmarks.csv',
                                           root_dir='faces/',
                                           transform=transforms.Compose([
                                               Rescale(256),
                                               RandomCrop(224),
                                               ToTensor()
                                           ]))

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['image'].size(), sample['landmarks'].size())

    if i == 3:
        break

现在可以使用dataloader进行数据迭代了：

In [ ]:
dataloader = DataLoader(transformed_dataset, batch_size=4,
                        shuffle=True, num_workers=0)


# 显示一批的数据
def show_landmarks_batch(sample_batched):
    """显示一批样本的图片和标记点."""
    images_batch, landmarks_batch = \
            sample_batched['image'], sample_batched['landmarks']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)

    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))

    for i in range(batch_size):
        plt.scatter(landmarks_batch[i, :, 0].numpy() + i * im_size,
                    landmarks_batch[i, :, 1].numpy(),
                    s=10, marker='.', c='r')

        plt.title('Batch from dataloader')

for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched['image'].size(),
          sample_batched['landmarks'].size())

    # 观察到第4批的时候就停止.
    if i_batch == 3:
        plt.figure()
        show_landmarks_batch(sample_batched)
        plt.axis('off')
        plt.ioff() # 1
        plt.show()
        break

---
以上内容就是本次对pandas数据预处理以及PyTorch数据加载的简单介绍，感谢大家的阅读。
有内容上的错误或疑问可以联系作者进行讨论：1700012810@pku.edu.cn。